## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string
from gensim.models.fasttext import FastText
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
stoplist = stopwords.words("russian") + list(string.punctuation) + ["«", "»"]
sb = SnowballStemmer(language="russian")

In [2]:
df_test = pd.read_csv("../data/news_test.txt", sep="\t", header=None)
df_test.columns = ["label", "title", "text"]

In [3]:
df_train = pd.read_csv("../data/news_train.txt", sep="\t", header=None)
df_train.columns = ["label", "title", "text"]

# обработка с помощью pymorphy2

In [4]:
def stop_and_stem(text_list):
    res = []
    for word in text_list:
        if word not in stoplist:
            res.append(morph.parse(word)[0].normal_form.replace(".", ""))
    return res

In [5]:
df_train['tokens'] = df_train["text"].apply(lambda x: word_tokenize(x, language="russian"))
df_train["lem"] = df_train['tokens'].apply(stop_and_stem)

In [6]:
df_test["tokens"] = df_test["text"].apply(lambda x: word_tokenize(x, language="russian"))
df_test["lem"] = df_test['tokens'].apply(stop_and_stem)

# обучение модели fastText

In [7]:
model = FastText()
model.build_vocab(sentences=df_train["lem"])

In [8]:
model.train(sentences=df_train["lem"], total_examples=len(df_train["lem"]), epochs=20, workers=2)

In [10]:
wv = model.wv

In [11]:
wv.most_similar(positive=["санкт-петербург", "сша"], negative=["россия"], topn=5)

[('нью-йоркский', 0.5693014860153198),
 ('нью-йорк', 0.5630447864532471),
 ('город', 0.5014321208000183),
 ('йорк', 0.49309042096138),
 ('сан-франциско', 0.4904010593891144)]

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [13]:
df_train["text_lem"] = df_train["lem"].apply(lambda x: ' '.join(x))

In [14]:
df_train["text_lem"][0]

'нападать вашингтон кэпиталзти александр овечкин передать детский хоккейный школа автомобиль получить окончание матч звезда национальный хоккейный лига нхл о сообщаться официальный сайт лигиавтомобиль honda accord подарить хоккеист решение спонсор мероприятие игрок нхл пожертвовать машина спортивный школа nova cool cats special hockey inc который расположить штат вирджинияовечкин общаться 10-летний девочка анна чтоб синдром даун который заниматься школа являться поклонница спортсмен в сентябрь форвард пообедать вместе юный хоккеистка японский ресторанематч звезда нхл коламбус штат огайо завершиться победа команда джонатан тэйвз команда ник фолиньо счёт 17:12 овечкин выступать проиграть коллектив россиянин отметиться три результативный передача'

In [15]:
df_train['word_count'] = df_train['lem'].apply(lambda x: len(x))

In [16]:
df_train['word_count'].describe()

count    15000.000000
mean       152.139067
std         51.465810
min          3.000000
25%        117.000000
50%        145.000000
75%        179.000000
max        832.000000
Name: word_count, dtype: float64

# векторизация с помощью tf-idf и PCA

In [17]:
vectorizer = TfidfVectorizer(max_features=2 ** 11)
X = vectorizer.fit_transform(df_train["text_lem"])

In [18]:
df_test["text_lem"] = df_test["lem"].apply(lambda x: ' '.join(x))
X_test = vectorizer.transform(df_test["text_lem"])

In [19]:
X.shape

(15000, 2048)

In [20]:
pca = PCA(n_components=0.95, random_state=42)
X_reduced = pca.fit_transform(X.toarray())
X_reduced.shape

(15000, 1657)

In [21]:
X_test_reduced = pca.transform(X_test.toarray())

In [22]:
X_test.shape

(3000, 2048)

In [23]:
Encoder = LabelEncoder()
train_Y = Encoder.fit_transform(df_train["label"])
test_Y = Encoder.fit_transform(df_test["label"])

## Наивный байесовский классификатор

In [24]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(X, train_Y)

predictions_NB = Naive.predict(X_test)

print("NB accuracy: ", accuracy_score(predictions_NB, test_Y))

NB accuracy:  0.841


## Логистическая регрессия с подбором параметров

In [27]:
hyper = {'C' : [1, 4, 10, 100],
         'solver': ['lbfgs']}
gd=GridSearchCV(estimator=
                LogisticRegression(multi_class='multinomial', random_state=17, n_jobs=4),
                param_grid=hyper)
gd.fit(X_reduced, train_Y)
print(gd.best_score_)
print(gd.best_estimator_)

0.858
LogisticRegression(C=4, multi_class='multinomial', n_jobs=4, random_state=17)


In [28]:
# lr = LogisticRegression(multi_class='multinomial', random_state=17)
lr = gd.best_estimator_
lr.fit(X_reduced, train_Y)
predictions_lr = lr.predict(X_test_reduced)

print("LR accuracy -> ", accuracy_score(predictions_lr, test_Y))

LR accuracy ->  0.8696666666666667
